### 1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features. Use the data from 2005.


In [4]:
# prep: import modules and get pwd
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [75]:
# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''select f.film_id,
f.rental_duration,
f.rental_rate,
f.length,
f.replacement_cost,
f.rating,
f.special_features
from film as f
left join inventory as i 
on f.film_id = i.film_id
join rental as r
on r.inventory_id = i.inventory_id
where year(rental_date)=2005;'''

raw_features = pd.read_sql_query(query, engine)
raw_features.head()

,film_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features
0,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes"
1,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes"
2,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes"
3,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes"
4,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes"


In [93]:
# drop duplicates 
df_features = raw_features.copy()
df_features.drop_duplicates(inplace=True,ignore_index=True)
df_features.head()

,film_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features
0,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes"
1,2,3,4.99,48,12.99,G,"Trailers,Deleted Scenes"
2,3,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes"
3,4,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes"
4,5,6,2.99,130,22.99,G,Deleted Scenes


### 2.Create a query to get the total amount of rentals in June for each film.


In [78]:
query= '''select f.film_id,count(r.rental_id)
from film as f
join inventory as i
on f.film_id = i.film_id
join rental as r
on i.inventory_id = r.inventory_id
where r.rental_date >= '2005-06-01' AND r.rental_date < '2005-07-01'
group by f.film_id;'''

num_rentals_june = pd.read_sql_query(query, engine)
num_rentals_june.head()

,film_id,count(r.rental_id)
0,356,4
1,961,5
2,614,4
3,369,5
4,220,4


### 3. Do the same with July.


In [79]:
query= '''select f.film_id,count(r.rental_id)
from film as f
join inventory as i
on f.film_id = i.film_id
join rental as r
on i.inventory_id = r.inventory_id
where r.rental_date >= '2005-07-01' AND r.rental_date < '2005-08-01'
group by f.film_id;'''

num_rentals_july = pd.read_sql_query(query, engine)
num_rentals_july.head()

,film_id,count(r.rental_id)
0,194,7
1,376,11
2,184,9
3,600,5
4,19,8


### 4. Create a new column containing (Yes/No) for each film whether or not the number of monthly rentals in July was bigger than in June. Your objective will be to predict this new column.

In [80]:
# merge June and July rentals into one dataframe called num_rentals
num_rentals = num_rentals_june.merge(num_rentals_july,how='outer',on='film_id')
num_rentals

,film_id,count(r.rental_id)_x,count(r.rental_id)_y
0,356,4.0,11
1,961,5.0,7
2,614,4.0,11
3,369,5.0,13
4,220,4.0,13
...,...,...,...
953,485,NaN,3
954,939,NaN,3
955,904,NaN,1
956,400,NaN,2


In [81]:
# check missing value
num_rentals.isna().sum()

film_id                  0
count(r.rental_id)_x    58
count(r.rental_id)_y     0
dtype: int64

In [82]:
# deal with missing value. since in June, missing values mean no films are rented in June, I replace NaN with 0.
num_rentals = num_rentals.fillna(0)
num_rentals

,film_id,count(r.rental_id)_x,count(r.rental_id)_y
0,356,4.0,11
1,961,5.0,7
2,614,4.0,11
3,369,5.0,13
4,220,4.0,13
...,...,...,...
953,485,0.0,3
954,939,0.0,3
955,904,0.0,1
956,400,0.0,2


In [83]:
num_rentals=num_rentals.rename(columns={'count(r.rental_id)_x':'June','count(r.rental_id)_y':'July'})
num_rentals

,film_id,June,July
0,356,4.0,11
1,961,5.0,7
2,614,4.0,11
3,369,5.0,13
4,220,4.0,13
...,...,...,...
953,485,0.0,3
954,939,0.0,3
955,904,0.0,1
956,400,0.0,2


In [84]:
# generate a new column called target which I will predict.
num_rentals['target']=num_rentals['July'] > num_rentals['June']
num_rentals

,film_id,June,July,target
0,356,4.0,11,True
1,961,5.0,7,True
2,614,4.0,11,True
3,369,5.0,13,True
4,220,4.0,13,True
...,...,...,...,...
953,485,0.0,3,True
954,939,0.0,3,True
955,904,0.0,1,True
956,400,0.0,2,True


In [85]:
# replace True and False with Yes and No
num_rentals['target'] = num_rentals['target'].replace({True:'Yes',False:'No'})
num_rentals['target']

0      Yes
1      Yes
2      Yes
3      Yes
4      Yes
      ... 
953    Yes
954    Yes
955    Yes
956    Yes
957    Yes
Name: target, Length: 958, dtype: object

In [86]:
# Drop two redundant columns for further data frame merge
num_rentals = num_rentals.drop(['June','July'],axis=1)
num_rentals

,film_id,target
0,356,Yes
1,961,Yes
2,614,Yes
3,369,Yes
4,220,Yes
...,...,...
953,485,Yes
954,939,Yes
955,904,Yes
956,400,Yes


### 5. Read the data into a Pandas dataframe.


In [ ]:
# done before

### 6. Analyze extracted features and transform them. You may need to encode some categorical variables or scale numerical variables.


In [101]:
data = df_features.merge(num_rentals, how='outer',on='film_id')
data.head()

,film_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,target
0,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",Yes
1,2,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",Yes
2,3,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",Yes
3,4,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",Yes
4,5,6,2.99,130,22.99,G,Deleted Scenes,Yes


In [103]:
data.isna().sum()

film_id             0
rental_duration     0
rental_rate         0
length              0
replacement_cost    0
rating              0
special_features    0
target              0
dtype: int64

In [ ]:
#Xy split

In [112]:
X = data.drop(['target'],axis=1)
y = data['target']

In [113]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

In [ ]:
# deal with independent variables

In [121]:
#select numercial and categorical columns in the train set seperately
X_train_num = X_train.select_dtypes('number')
X_test_num = X_test.select_dtypes('number')

In [ ]:
# scaling numerical variables

In [118]:
from sklearn.preprocessing import StandardScaler
import numpy as np

In [134]:
transformer = StandardScaler().fit(X_train_num)
X_train_scaled_np = transformer.transform(X_train_num)
X_test_scaled_np = transformer.transform(X_test_num)

X_train_scaled = pd.DataFrame(X_train_scaled_np,columns=X_train_num.columns)
X_test_scaled = pd.DataFrame(X_test_scaled_np,columns =X_test_num.columns)

In [135]:
X_train_scaled.head()

,film_id,rental_duration,rental_rate,length,replacement_cost
0,-0.374052,0.744651,-0.020743,0.803132,0.967402
1,0.802146,-1.389025,-1.242962,-1.701852,1.459454
2,-0.054848,-1.389025,-0.020743,1.429378,1.623472
3,0.018014,0.033426,-1.242962,-1.701852,0.967402
4,0.011075,-0.677800,-0.020743,-1.075606,0.311333


In [136]:
X_test_scaled.head()

,film_id,rental_duration,rental_rate,length,replacement_cost
0,0.819494,1.455877,-0.020743,-0.299061,1.131420
1,1.312178,-0.677800,-1.242962,-1.250955,0.639368
2,0.129041,-1.389025,-0.020743,-1.225905,-0.180719
3,1.468311,1.455877,1.201477,-0.173812,0.475350
4,0.139450,1.455877,-0.020743,0.953431,0.311333


In [128]:
# covert categorical to numerical variables
from sklearn.preprocessing import OneHotEncoder
X_train_categorical = X_train.select_dtypes('object')
X_test_categorical =  X_test.select_dtypes('object')

In [129]:
encoder = OneHotEncoder() 
encoder.fit(X_train_categorical)

X_train_cat_np = encoder.transform(X_train_categorical).toarray()
X_test_cat_np = encoder.transform(X_test_categorical).toarray()

X_train_cat = pd.DataFrame(X_train_cat_np, columns = encoder.get_feature_names_out())

X_test_cat = pd.DataFrame(X_test_cat_np, columns = encoder.get_feature_names_out())


X_train_cat.head()

,rating_G,rating_NC-17,rating_PG,rating_PG-13,rating_R,special_features_Behind the Scenes,special_features_Commentaries,"special_features_Commentaries,Behind the Scenes","special_features_Commentaries,Deleted Scenes","special_features_Commentaries,Deleted Scenes,Behind the Scenes",special_features_Deleted Scenes,"special_features_Deleted Scenes,Behind the Scenes",special_features_Trailers,"special_features_Trailers,Behind the Scenes","special_features_Trailers,Commentaries","special_features_Trailers,Commentaries,Behind the Scenes","special_features_Trailers,Commentaries,Deleted Scenes","special_features_Trailers,Commentaries,Deleted Scenes,Behind the Scenes","special_features_Trailers,Deleted Scenes","special_features_Trailers,Deleted Scenes,Behind the Scenes"
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 7. Create a logistic regression model to predict this new column from the cleaned data.


In [140]:
from sklearn.linear_model import LogisticRegression

In [153]:
# merge X_train_scaled and X_train_cat
X_train_transformed = np.concatenate([X_train_scaled, X_train_cat], axis=1)

In [154]:
log_reg= LogisticRegression(random_state=0, solver='saga').fit(X_train_transformed, y_train)

In [155]:
#merge X_test_scaled and X_test_cat
X_test_transformed = np.concatenate([X_test_scaled, X_test_cat], axis=1)


In [159]:
y_pred_test = log_reg.predict(X_test_transformed)

### 8. Evaluate the results.


In [165]:
log_reg.score(X_test_transformed, y_test)

0.9947916666666666